# KNN

In [20]:
import pandas as pd
df=pd.read_csv("ml_data.csv")

In [21]:
df

Age  Gender  BMI  Medications  VitaminA  VitaminC  Fat  Cholesterol  \
0    34       1   24            2         0         1    2            5   
1    45       0   22            1         1         0    2            5   
2    25       1   26            3         0         1    1            3   
3    55       0   28            4         1         0    1            4   
4    33       1   23            5         0         1    2            5   
..  ...     ...  ...          ...       ...       ...  ...          ...   
95   20       0   53            4         0         1    2            5   
96   46       1   21            2         0         1    2            5   
97   32       0   27            1         1         0    2            5   
98   47       1   25            3         0         1    1            3   
99   30       0   29            4         1         0    1            4   

    Sodium  Carbohydrate  Protein  Calcium_mg  Sugar_g  Iron_mg  Calories  \
0       20             1       30         300        5        3        50   
1       20             1       30         350       10        4       150   
2       15             2       20         400       15        5       100   
3       10             1       15         450        5        6        95   
4       20             2       30         500       10        3       250   
..     ...           ...      ...         ...      ...      ...       ...   
95      20             2       30         500       10        6        95   
96      20             1       30         350       10        3        55   
97      20             2       30         400        5        4       160   
98      15             1       20         450       10        5       110   
99      10             2       15         500        5        6       100   

    Category  Product_ID  
0          3          10  
1          1           6  
2          3           4  
3          4           9  
4          2           5  
..       ...         ...  
95         1           5  
96         3          10  
97         1           6  
98         4           3  
99         2           9  

[100 rows x 17 columns]

In [28]:
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Assume you have a pandas dataframe `df` containing the dataset
X = df.drop("Product_ID", axis=1)
y = df["Product_ID"]

# Scaling the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2)

# Training the KNN model
k = 5
knn = NearestNeighbors(n_neighbors=k, algorithm='auto', metric='euclidean')
knn.fit(X_train, y_train)


NearestNeighbors(metric='euclidean')

In [24]:
# import numpy as np

# def mean_reciprocal_rank(y_true, recommended_products):
#     mrr = 0
#     n = len(y_true)
#     for i in range(n):
#         correct_product = y_true[i]
#         if correct_product in recommended_products[i]:
#             rank = np.where(recommended_products[i] == correct_product)[0][0] + 1
#             mrr += 1 / rank
#     return mrr / n

# # Get the recommended products for all test instances
# y_true = y_test.values
# recommended_products_all = []

# for i in range(len(X_test)):
#     query = X_test[i].reshape(1, -1)
#     distances, indices = knn.kneighbors(query)
#     recommended_products = y_train.iloc[indices.flatten()].values
#     recommended_products_unique = np.unique(recommended_products)
#     recommended_products_all.append(recommended_products_unique)

# # Compute MRR
# mrr = mean_reciprocal_rank(y_true, recommended_products_all)
# print("Mean Reciprocal Rank:", mrr)


In [38]:
# Get the indices and distances of the 5 nearest neighbors for a given input
query = X_test[0].reshape(1, -1)  # Replace this with any input query
distances, indices = knn.kneighbors(query)


# Get the Product_IDs of the recommended products and remove duplicates
recommended_products_dict = {x: i for i, x in enumerate(y_train.iloc[indices.flatten()].values)}
recommended_products = [x for x in recommended_products_dict.keys()]
# Remove duplicate rows from the dataframe
recommended_products_df.drop_duplicates(subset='Product ID', keep='first', inplace=True)
# Get the product information for the recommended products
recommended_products_df = recommended_products_df.loc[recommended_products_df['Product ID'].isin(recommended_products)]
# Sort the dataframe by the order of the recommended products
recommended_products_df['rank'] = recommended_products_df['Product ID'].apply(lambda x: recommended_products_dict[x])
recommended_products_df.sort_values('rank', inplace=True)
recommended_products_df.drop('rank', axis=1, inplace=True)



# Sort the dataframe by the order of the recommended products
recommended_products_df.set_index('Product ID', inplace=True)
recommended_products_df = recommended_products_df.reindex(recommended_products)
recommended_products_df.reset_index(inplace=True)


recommended_products_df

Product ID Product Name                                           Features
0          10  VitalGreens  Contains Vitamins A, C, E, and K; Contains 50K...
1           6    NutriGlow  Contains Vitamin A, Vitamin E, and Selenium; C...
2           2   DigestEase  Contains Probiotics, Prebiotics, and Digestive...
3           9  VisionGuard  Contains Lutein, Zeaxanthin, and Vitamin A; Co...

# Rule Based

In [36]:
import pandas as pd

def rule_based_classifier(data, user_preferences):
    results = data.copy()

    # Filter by VitaminA and VitaminC
    results = results[(results['VitaminA'] == user_preferences['vitamin_a']) &(results['VitaminC'] == user_preferences['vitamin_c'])]

    # Filter by Fat, Cholesterol, Sodium, Carbohydrate, Protein, Calcium_mg, Sugar_g, Iron_mg, and Calories
    for key, value_range in user_preferences['ranges'].items():
        results = results[(results[key] >= value_range[0]) & (results[key] <= value_range[1])]
    # Filter by Contains
    if user_preferences['exclude_ingredients']:
        for ingredient in user_preferences['exclude_ingredients']:
            results = results[~results['Contains'].str.contains(ingredient)]
    # print(results)
    # Filter by Safe and Not_recommended
    results = results[(results['Safe'].str.contains(user_preferences['health_condition'])) | (~results['Not_recommended'].str.contains(user_preferences['health_condition']))]

    return results


In [54]:
import pandas as pd

# Load your dataset
data = pd.read_csv("final_products.csv")

# # Preprocess dataset
# data.dropna(inplace=True)
data['Contains'] = data['Contains'].str.lower()
data.head()

Product_ID         Name                          Safe Not_recommended  \
0         1.0   AdrenalAid         adrenal_health;stress            none   
1         2.0      AgeDefy        anti-aging;skin_health            none   
2         3.0  AlkaDefense        alkaline_balance;detox            none   
3         4.0  AquaHydrate        hydration;electrolytes            none   
4         5.0    BabyBelly  pregnancy_nausea;indigestion            none   

   VitaminA  VitaminC  Fat  Cholesterol  Sodium  Carbohydrate  Protein  \
0       0.0       1.0  3.0          5.0    15.0           3.0     30.0   
1       1.0       1.0  1.0          5.0    15.0           3.0     20.0   
2       0.0       0.0  2.0          5.0    20.0           4.0     15.0   
3       0.0       1.0  0.0          0.0     5.0           2.0      0.0   
4       0.0       1.0  5.0         10.0    30.0           2.0     45.0   

   Calcium_mg  Sugar_g  Iron_mg  Calories                          Contains  \
0       120.0      5.0      2.0      50.0      ashwagandha;rhodiola;ginseng   
1       150.0      7.0      1.0      55.0       resveratrol;coq10;vitamin_c   
2        80.0      6.0      2.0      40.0  spirulina;chlorella;barley_grass   
3         0.0      0.0      1.0      15.0           electrolytes;b-vitamins   
4       180.0      7.0      5.0      75.0      ginger;vitamin_b6;peppermint   

  Rating  N_reviews  Review_Score  
0    2.5       52.0           NaN  
1    4.8       93.0           NaN  
2    3.9       15.0           NaN  
3    3.4       72.0           NaN  
4    1.6       61.0           NaN

In [55]:
# User preferences
user_preferences = {
    'vitamin_a': 1,
    'vitamin_c': 1,
    'ranges': {
        'Fat': (1, 5),
        'Cholesterol': (5, 20),
        'Sodium': (5, 25),
        'Carbohydrate': (1, 40),
        'Protein': (20, 150),
        'Calcium_mg': (100, 200),
        'Sugar_g': (3, 7),
        'Iron_mg': (1, 4),
        'Calories': (40, 70)
    },
    'exclude_ingredients': ['Ginseng', 'Omega-3'],
    'health_condition': 'stress'
}

# Calculate the weighted score
rating_weight = 0.6
reviews_weight = 1 - rating_weight

# Make sure exclude_ingredients are in lowercase
user_preferences['exclude_ingredients'] = [ingredient.lower() for ingredient in user_preferences['exclude_ingredients']]

# Convert the 'Rating' and 'N_reviews' columns to numeric types
df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce')
df['N_reviews'] = pd.to_numeric(df['N_reviews'], errors='coerce')

df['Weighted_Score'] = (df['Rating'] * rating_weight) + (df['N_reviews'] * reviews_weight / 100)

# Sort the DataFrame by the weighted score
df = df.sort_values(by='Weighted_Score', ascending=False)

# Call the rule_based_classifier function
recommended_products = rule_based_classifier(df, user_preferences)
recommended_products


Product_ID          Name                                Safe  \
1          2.0       AgeDefy              anti-aging;skin_health   
33        34.0    HairGrowth             hair_loss;brittle_nails   
7          8.0  BoneStrength  lactose_intolerance;kidney_disease   
51        52.0  MightyMarrow              osteoporosis;fractures   
12        13.0  CardioCharge         heart_health;blood_pressure   
30        31.0   GutGuardian                digestion;gut_health   
59        60.0  PrenatalPlus         pregnancy;fetal_development   
66        67.0   SkinRadiant              skin_health;complexion   
31        32.0     GutRelief                gut_health;digestion   
26        27.0    FlexiJoint                joint_pain;arthritis   
32        33.0    Hairfinity                 hair_loss;dry_scalp   
57        58.0     NutriGlow          heart_conditions;diabetics   
29        30.0  GlucoControl       blood_sugar_support;diabetics   
10        11.0   BreatheFree                    asthma;allergies   

   Not_recommended  VitaminA  VitaminC  Fat  Cholesterol  Sodium  \
1             none       1.0       1.0  1.0          5.0    15.0   
33            none       1.0       1.0  2.0          5.0    20.0   
7             none       1.0       1.0  4.0          8.0    25.0   
51            none       1.0       1.0  4.0          8.0    25.0   
12    hypertension       1.0       1.0  2.0          6.0    20.0   
30            none       1.0       1.0  3.0          6.0    25.0   
59            none       1.0       1.0  4.0          8.0    25.0   
66            none       1.0       1.0  1.0          5.0    20.0   
31            none       1.0       1.0  3.0          8.0    20.0   
26            none       1.0       1.0  3.0          6.0    20.0   
32            none       1.0       1.0  2.0          5.0    20.0   
57            none       1.0       1.0  2.0          5.0    20.0   
29            none       1.0       1.0  1.0          6.0    25.0   
10            none       1.0       1.0  1.0          5.0    15.0   

    Carbohydrate  Protein  Calcium_mg  Sugar_g  Iron_mg  Calories  \
1            3.0     20.0       150.0      7.0      1.0      55.0   
33           1.0     30.0       150.0      5.0      3.0      55.0   
7            1.0     35.0       200.0      3.0      4.0      70.0   
51           1.0     35.0       200.0      3.0      4.0      70.0   
12           2.0     25.0       100.0      5.0      2.0      45.0   
30           3.0     40.0       180.0      6.0      4.0      70.0   
59           4.0     40.0       200.0      4.0      4.0      60.0   
66           2.0     20.0       100.0      7.0      1.0      45.0   
31           3.0     30.0       150.0      7.0      4.0      60.0   
26           2.0     30.0       150.0      6.0      4.0      55.0   
32           1.0     30.0       150.0      5.0      3.0      55.0   
57           1.0     30.0       150.0      5.0      3.0      55.0   
29           2.0     20.0       110.0      5.0      1.0      50.0   
10           3.0     20.0       200.0      4.0      3.0      50.0   

                               Contains  Rating  N_reviews  Review_Score  \
1           resveratrol;coq10;vitamin_c     4.8       93.0           NaN   
33              biotin;keratin;collagen     4.8       60.0           NaN   
7         calcium;vitamin_d3;vitamin_k2     4.5       87.0           NaN   
51            magnesium;vitamin_d;boron     4.1       89.0           NaN   
12            beetroot;l-arginine;coq10     4.3        3.0           NaN   
30       probiotics;enzymes;l-glutamine     3.4       59.0           NaN   
59                  folic_acid;iron;dha     2.3       92.0           NaN   
66            biotin;collagen;vitamin_e     1.6       78.0           NaN   
31    probiotics;prebiotics;l-glutamine     1.7       42.0           NaN   
26          glucosamine;chondroitin;msm     1.8       22.0           NaN   
32         msm;silica;horsetail_extract     1.3       92.0           NaN   
57         vit

In [43]:
df

Empty DataFrame
Columns: [Product_ID, Name, Safe, Not_recommended, VitaminA, VitaminC, Fat, Cholesterol, Sodium, Carbohydrate, Protein, Calcium_mg, Sugar_g, Iron_mg, Calories, Contains, Rating, N_reviews, Review_Score, Weighted_Score]
Index: []

In [29]:
import pandas as pd
import numpy as np

# Generate random data
np.random.seed(42)
data = {
    'Number_of_Reviews': np.random.randint(1, 100, 80)
}

# Create a DataFrame
df = pd.DataFrame(data)

# Print the DataFrame in a table format
print(df.to_string(index=False))


 Number_of_Reviews
                52
                93
                15
                72
                61
                21
                83
                87
                75
                75
                88
                24
                 3
                22
                53
                 2
                88
                30
                38
                 2
                64
                60
                21
                33
                76
                58
                22
                89
                49
                91
                59
                42
                92
                60
                80
                15
                62
                62
                47
                62
                51
                55
                64
                 3
                51
                 7
                21
                73
                39
                18
                 4
            

In [5]:
class DAWGNode:
    def __init__(self):
        self.edges = {}

    def __str__(self):
        return str(self.edges)

class DAWG:
    def __init__(self):
        self.root = DAWGNode()

    def insert(self, word):
        current_node = self.root
        for letter in word:
            if letter not in current_node.edges:
                current_node.edges[letter] = DAWGNode()
            current_node = current_node.edges[letter]

        current_node.edges['$'] = None

    def __str__(self):
        return str(self.root)    
    def search(self, word):
        current_node = self.root
        for letter in word:
            if letter not in current_node.edges:
                return False
            current_node = current_node.edges[letter]

        return '$' in current_node.edges

# ...

def build_dawg_serial(words):
    dawg = DAWG()
    for word in words:
        dawg.insert(word)
    return dawg

words = ["apple", "app", "bat", "batman", "ball"]
# dawg = build_dawg_serial(words)
# print(dawg)

dawg_serial = build_dawg_serial(words)
print(dawg_serial.search("apple"))  # True
print(dawg_serial.search("banana"))  # False


True
False


In [6]:
import concurrent.futures

class DAWGNode:
    def __init__(self):
        self.edges = {}

    def __str__(self):
        return str(self.edges)

class DAWG:
    def __init__(self):
        self.root = DAWGNode()

    def insert(self, word):
        current_node = self.root
        for letter in word:
            if letter not in current_node.edges:
                current_node.edges[letter] = DAWGNode()
            current_node = current_node.edges[letter]

        current_node.edges['$'] = None

    def __str__(self):
        return str(self.root)

    def search(self, word):
        current_node = self.root
        for letter in word:
            if letter not in current_node.edges:
                return False
            current_node = current_node.edges[letter]

        return '$' in current_node.edges

def search_words_parallel(dawg, words_to_search, num_workers):
    search_results = {}

    def worker(word):
        search_results[word] = dawg.search(word)

    with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
        executor.map(worker, words_to_search)

    return search_results


def build_dawg_serial(words):
    dawg = DAWG()
    for word in words:
        dawg.insert(word)
    return dawg

def build_dawg_parallel(words, num_workers):
    dawg = DAWG()

    def worker(word):
        dawg.insert(word)

    with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
        executor.map(worker, words)

    return dawg

words = ["apple", "app", "bat", "batman", "ball"]
num_workers = 4

dawg_serial = build_dawg_serial(words)


In [7]:
# ...
dawg_parallel = build_dawg_parallel(words, num_workers)

words_to_search = ["apple", "banana", "batman", "orange"]
search_results = search_words_parallel(dawg_parallel, words_to_search, num_workers)

for word, result in search_results.items():
    print(f"{word}: {'found' if result else 'not found'}")


apple: found
banana: not found
batman: found
orange: not found
